## Extracting the data form the daily trending music videos 

#### Using Python,selenium, requests module & Youtube API I have tried to get the data & stats of the daily trending music videos in India. Further this data is returned as a dataframa.  The output can be further stored & sent as a csv


In [1]:
import requests
import pandas as pd
import time
import os

from selenium import webdriver 
from selenium.webdriver.common.by import By 
from selenium.webdriver.support.ui import WebDriverWait 
from selenium.webdriver.support import expected_conditions as EC

from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.by import By

### Giving the input-parameters required

In [3]:
# Api-Key from google devmode,Youll have to get your api Key from the Google developers console, 
# link for the same has been provided below for further info on this you can refer my blog

API_Key = "Get your API Key from - https://console.cloud.google.com/apis/dashboard"
           
API_Key = "Lpao4sdDh5hDh5k78Dcafnee87fae9fhae8fnae" #--you Key will look like this
pageToken = ""

#path of the chrome webdriver local
chrome_path = r"C:\SeleniumDriver\chromedriver.exe"

#### Connecting to chrome web-driver , getting the urls for trending music videos in India

In [4]:
#we're connecting to the Chromedriver for selenium to get the html response

s=Service(ChromeDriverManager().install())
driver = webdriver.Chrome(service=s)
driver.maximize_window()



====== WebDriver manager ======
Current google-chrome version is 100.0.4896
Get LATEST chromedriver version for 100.0.4896 google-chrome
Driver [C:\Users\ranap\.wdm\drivers\chromedriver\win32\100.0.4896.60\chromedriver.exe] found in cache


In [5]:
##getting the urls of the trnding music videos
driver.get("https://www.youtube.com/feed/trending?bp=4gINGgt5dG1hX2NoYXJ0cw%3D%3D")
user_data = driver.find_elements(by=By.XPATH,value = '//ytd-video-renderer/div//h3/a') ##fetch urls by X.path
trending_video_links = []
for i in user_data:
    trending_video_links.append(i.get_attribute('href'))

print("\nNo of videos on Trending",len(trending_video_links))
print("\nurl examples")
print(trending_video_links[:5])


No of videos on Trending 30

url examples
['https://www.youtube.com/watch?v=Tc8kb5HBNrA', 'https://www.youtube.com/watch?v=QFTt0kFcq18', 'https://www.youtube.com/watch?v=QHJstKwI0Ic', 'https://www.youtube.com/watch?v=fBWUSK-AedE', 'https://www.youtube.com/watch?v=Y40iZmuylKk']


### Getting the relevant data & stats of the trending videos using the Youtube API

In [6]:
##we'll define a function to clean urls for video id of each video & get the response
def get_response_data (urls):
    response_data=[]
    for url in urls:
        
        videoid = url.split("watch?v=")[-1].split("&list")[0]

        #URL to get the data/stats of Any video on youtube
        videostats_url = "https://www.googleapis.com/youtube/v3/videos?id="+videoid+"&part=snippet,statistics&key="+API_Key
        response_data.append(requests.get(videostats_url).json())
        
    return response_data


response_data = get_response_data(trending_video_links)

In [7]:
# len(response_data)

In [8]:
#Here from the json returned by the sending the request,we'll fetch the required data that's relevant for us

title = [];channel=[]; released=[]; viewcount=[]; likecount=[]; commentscount=[];
for video_data in response_data:
    if video_data['items'][0]['kind'] ==  "youtube#video":
        
        title .append(video_data['items'][0]['snippet']['title'])

        channel.append(video_data['items'][0]['snippet']['channelTitle'])

        released .append(video_data['items'][0]['snippet']['publishedAt'])

        viewcount .append(video_data['items'][0]['statistics']['viewCount'])

        likecount .append(video_data['items'][0]['statistics']['likeCount'])

        commentscount .append(video_data['items'][0]['statistics']['commentCount'])
        

### Returning the data & stats in form of a Dataframe 

In [9]:
Videodf = pd.DataFrame({"Title":title,"Channel Name":channel,
                        "Release Date-Time":released,"Views":viewcount,"Likes":likecount,
                        "No of Comments":commentscount,"url":trending_video_links})

Videodf

,Title,Channel Name,Release Date-Time,Views,Likes,No of Comments,url
0,Video: Mast Nazron Se | Rochak K ft Jubin Naut...,T-Series,2022-03-31T05:30:10Z,15321051,806774,44595,https://www.youtube.com/watch?v=Tc8kb5HBNrA
1,Big Man (Vadde Bande) - R Nait (Official Video...,R Nait,2022-03-29T04:32:15Z,12646536,404339,13571,https://www.youtube.com/watch?v=QFTt0kFcq18
2,Mera Yaar (Full Video) LEKH | Gurnam Bhullar |...,White Hill Music,2022-03-25T04:30:15Z,5792508,242822,8659,https://www.youtube.com/watch?v=QHJstKwI0Ic
3,Harrdy Sandhu - Kudiyan Lahore Diyan | Aisha S...,DM - Desi Melodies,2022-03-30T06:33:12Z,10835159,312651,82590,https://www.youtube.com/watch?v=fBWUSK-AedE
4,CHAK CHAK CHAK : Khan Bhaini Ft Shipra Goyal |...,BANG Music,2022-03-22T11:30:14Z,9931821,251736,5830,https://www.youtube.com/watch?v=Y40iZmuylKk
5,Shadow (Official Video) Jassa Dhillon | Gur Si...,Brown Town Music,2022-03-29T03:30:02Z,5871801,161993,6479,https://www.youtube.com/watch?v=9TuSIrYfomQ
6,VIDEO | फलाना बो फरार भईली | Pawan Singh Ft. S...,Pawan Singh Official,2022-03-13T01:15:08Z,12718050,367879,151426,https://www.youtube.com/watch?v=FFlHFAKKH50
7,Aashiqui Aa Gayi (Full Video) Radhe Shyam | Pr...,T-Series,2022-03-22T12:30:11Z,5056467,136179,2202,https://www.youtube.com/watch?v=lm3yy29vbqM
8,Kya Kar Diya (Official Video) Vishal Mishra | ...,VYRLOriginals,2022-03-24T05:30:02Z,11827257,203877,115357,https://www.youtube.com/watch?v=BctXfxyfF5s
9,Tere Ishq Mein Naachenge | Kumar Sanu | Alish...,Evergreen Bollywood Hits,2022-03-24T02:30:13Z,1695903,9099,169,https://www.youtube.com/watch?v=hrwCDy-DoSo


#### saving our file to a csv at a specific location

In [14]:
import os
Videodf.to_csv(os.path.join(r'C:\Users\ranap\OneDrive\Desktop\Crawling & API\Youtube Extraction',
                            r'Youtube trending music.csv'),index=False)